# Unit 02: Variables in classes

## Topics

* Object members
* Static members
* Style Guide
* Scopes
* Excercises

## Object members

* Object members are directly attached to the object
* All members are public
* If direct access is not desired the variable should be prefixed with '_' (see Style Guide)
* Access can be controlled using getters and setters

Example:

In [18]:
class MemberTest:
    def __init__(self, public_value, private_value):
        self.public_value = public_value
        self._private_value = private_value
    
    @property
    def private_value(self):
        return "*****"
    
a = MemberTest("my name", "my password")
b = MemberTest("another name", "very secret")

print("{} {}".format(a.public_value, a.private_value))
print("{} {}".format(b.public_value, b.private_value))

# BUT:
print(a._private_value)
print(b._private_value)

my name *****
another name *****
my password
very secret


## Static members

* Static members are attached to the class
    * And so shared across all instances of that class

Example:

In [14]:
class StaticTest:
    instances = 0
    
    def __init__(self, value):
        DummyClass.instances += 1
        
        self._value = value
    
    @property
    def value(self):
        return self._value
        
a = StaticTest("a")
b = StaticTest("b")
print(StaticTest.instances)
print("{} {}".format(a.value, a.instances))
print("{} {}".format(b.value, b.instances))

0
a 0
b 0


## Naming conventions

Style Guide for Python Code: https://www.python.org/dev/peps/pep-0008/

* Modules should have short, all-lowercase names.
* Class names should normally use the CapWords convention.
* Function names should be lowercase, with words separated by underscores as necessary.
* Method names should be lowercase, with words separated by underscores as necessary.
* Instance variable names should be lowercase, with words separated by underscores as necessary.
* Constants are usually defined on a module level and written in all capital letters with underscores separating words.
* _single_leading_underscore : weak "internal use" indicator.
* Always use self for the first argument to instance methods.
* Always use cls for the first argument to class methods.

Example:

In [3]:
SOME_CONSTANT = 3.14159265359
normal_variable = None

def a_function_name():
    pass

class SomeClass:
    CLASS_CONSTANT = 3.14159265359
    
    def a_method_name(self):
        pass

## Scope

Scopes define the lifetime of a variable and where they can be accessed from.

Loosely spoken there are only 2 scopes in Python:
* Global scope
    * Global variables are always visible/accessible
* Local scope
    * Enclosed in functions
    * Enclosed in methods
        * Values can only be shared when attached to the object (e.g. self.x)

In [36]:
test = 100
another_test = 101

for i in range(10):
    pass
print(i) # i still exists since it is in the global scope

def dummy_function():
    for j in range(10, 20):
        pass
    print(j) # j still exists (-> local scope)
    
    # print(test) # would cause an exception since we will change it in this scope
    test = 20
    print(test) # new instance of test created in function scope
    
    print(another_test)
    
dummy_function()
print(test) # test has still the same value in the global scope
# print(j) # j doesn't exist in the global scope

9
19
20
101
100


## Programming example 1

* Write an abstract Cache class
    * The method get(key) should retrieve a cached value
    * If the value is not cached, yet it should be retrieved with the retrieve(key) method
        * The implementation of the retrieve method should be left to the subclasses
* Subclass Cache
* Test it

In [19]:
from urllib import request

class Cache:
    """Simplest abstract cache implementation ever. Sublclasses must implement
    the retrieve(key) method."""

    def __init__(self):
        self._cache = {}

    def get(self, key):
        """Searches for a chached value for key. Returns the value if found.
        Otherwise the retrieve(key) method is executed, whose return value is
        cached for future calls."""

        if key in self._cache:
            value = self._cache[key]
        else:
            value = self.retrieve(key)
            self._cache[key] = value
        return value


class EvalCache(Cache):
    """Caching calls to eval. Consider avoiding calls to eval."""

    def __init__(self):
        Cache.__init__(self)

    def retrieve(self, key):
        """Evaluates key and returns it."""
        return eval(key)


class WebCache(Cache):
    """Caching web requests with
    https://docs.python.org/3/library/urllib.request.html#urllib.request.urlopen."""

    def __init__(self):
        Cache.__init__(self)

    def retrieve(self, key):
        """Retrieves the values to cache with urlopen."""
        with request.urlopen(key) as u:
            return u.read()


if __name__ == '__main__':
    cache = EvalCache()
    print(cache.get("4+5"))
    print(cache.get("4+5"))

    web = WebCache()
    web.get("http://www.google.com")
    web.get("http://www.google.com")


9
9


## Programming example 2

Write a class which represents a person.
* On initialization it should take a name
* Marry a Person with the marry method - in our world only unmarried Persons can marry
* Write a Method get_child
    * In our world only marrid persons can get children
    * The method first checks whether the person is married
    * Then it updates both the mother and father to be the parent
    * There should be a member variable to remember the parents.


In [6]:
class Person:
    def __init__(self, name):
        self._name = name
        self._married_to = None
        self._children = []
        self._parents = []
        
    @property
    def name(self):
        return self._name
        
    @property
    def married_to(self):
        return self._married_to
    
    @property
    def children(self):
        return self._children

    @property
    def parents(self):
        return self._parents
        
    def marry(self, other):
        if self._married_to:
            raise ValueError("only unmarried persons can marry")

        self._married_to = other
        other._married_to = self
        
    def get_child(self, child):
        if not self._married_to:
            raise ValueError("unmarried persons cant get children")
            
        self._children.append(child)
        self._married_to._children.append(child)
        
        child._parents.append(self)
        child._parents.append(self._married_to)
        
    def __str__(self):
        return "<{}>".format(self._name)
        
hans = Person("hans")
petra = Person("petra")
child = Person("child")

petra.marry(hans)
hans.get_child(child)

print(hans.married_to)
print(petra.married_to)

print(hans.children)
print(petra.children)

<petra>
<hans>
